# Fine-tune GlowTTS model with LJSpeech dataset Quick Tutorial.
Ref: https://docs.coqui.ai/en/dev/tutorial_for_nervous_beginners.html

In [1]:
# check cuda availability
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Wed_Feb__8_05:53:42_Coordinated_Universal_Time_2023
Cuda compilation tools, release 12.1, V12.1.66
Build cuda_12.1.r12.1/compiler.32415258_0


In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
import os
if os.getenv("CUDA_VISIBLE_DEVICES") is None:
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
import os
import re
from pathlib import Path

# Trainer: Where the ✨️ happens.
# TrainingArgs: Defines the set of arguments of the Trainer.
from trainer import Trainer, TrainerArgs

# GlowTTSConfig: all model related values for training, validating and testing.
from TTS.tts.configs.glow_tts_config import GlowTTSConfig

# BaseDatasetConfig: defines name, formatter and path of the dataset.
from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.glow_tts import GlowTTS
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor

# we use the same path as this script as our training folder.
project_dir = Path(re.search(r".*Cockatoo.AI", os.getcwd())[0])
data_path = project_dir / "experiments/model_c/data/ljspeech"
output_path = project_dir / "experiments/model_c/output"

In [5]:
# Download dataset if not yet exists
if "LJSpeech-1.1" not in map(lambda x: x.name, data_path.glob("*")):
    from TTS.utils.downloaders import download_ljspeech
    download_ljspeech(data_path)

In [6]:
# DEFINE DATASET CONFIG
# Set LJSpeech as our target dataset and define its path.
# You can also use a simple Dict to define the dataset and pass it to your custom formatter.
dataset_config = BaseDatasetConfig(
    formatter="ljspeech", meta_file_train="metadata.csv", path= (data_path / "LJSpeech-1.1/").as_posix()
)
display(dataset_config)

# INITIALIZE THE TRAINING CONFIGURATION
# Configure the model. Every config class inherits the BaseTTSConfig.
config = GlowTTSConfig(
    run_description="🐸Coqui trainer run.",
    batch_size=32,
    eval_batch_size=16,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=100,  # NOTE: mini run for demo. A complete 1000 run will be uploaded separately.
    text_cleaner="phoneme_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=(output_path / "phoneme_cache").as_posix(),
    print_step=25,
    print_eval=False,
    cudnn_enable=True,
    mixed_precision=True,
    output_path=output_path.as_posix(),
    datasets=[dataset_config],
)

BaseDatasetConfig(formatter='ljspeech', dataset_name='', path='c:/Users/wkaic/GitHub/Cockatoo.AI/experiments/model_c/data/ljspeech/LJSpeech-1.1', meta_file_train='metadata.csv', ignored_speakers=None, language='', phonemizer='', meta_file_val='', meta_file_attn_mask='')

In [7]:
# INITIALIZE THE AUDIO PROCESSOR
# Audio processor is used for feature extraction and audio I/O.
# It mainly serves to the dataloader and the training loggers.
ap = AudioProcessor.init_from_config(config)

# INITIALIZE THE TOKENIZER
# Tokenizer is used to convert text to sequences of token IDs.
# If characters are not defined in the config, default characters are passed to the config
tokenizer, config = TTSTokenizer.init_from_config(config)

# INITIALIZE THE MODEL
# Models take a config object and a speaker manager as input
# Config defines the details of the model like the number of layers, the size of the embedding, etc.
# Speaker manager is used by multi-speaker models.
model = GlowTTS(config, ap, tokenizer, speaker_manager=None)


# LOAD DATA SAMPLES
# Each sample is a list of ```[text, audio_file_path, speaker_name]```
# You can define your custom sample loader returning the list of samples.
# Or define your custom formatter and pass it to the `load_tts_samples`.
# Check `TTS.tts.datasets.load_tts_samples` for more details.
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:45
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 | > Found 13100 files in C:\Users\wkaic\GitHub\Cockatoo.AI\experiments\model_c\data\ljspeech\LJSpeech-1.1


In [8]:
# INITIALIZE THE TRAINER
# Trainer provides a generic API to train all the 🐸TTS models with all its perks like mixed-precision training,
# distributed training, etc.
trainer = Trainer(
    TrainerArgs(), config, output_path.as_posix(), model=model, train_samples=train_samples, eval_samples=eval_samples
)

# AND... 3,2,1... 🚀
trainer.fit()

 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: True
 | > Precision: fp16
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 20
 | > Num. of Torch Threads: 12
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=c:/Users/wkaic/GitHub/Cockatoo.AI/experiments/model_c/output\run-April-13-2024_09+45AM-046eb8b
c:\Users\wkaic\miniconda3\envs\cockatoo\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

 > Model has 28610257 parameters

 > EPOCH: 0/100
 --> c:/Users/wkaic/GitHub/Cockatoo.AI/experiments/model_c/output\run-April-13-2024_09+45AM-046eb8b




> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: en-us
		| > phoneme backend: gruut
| > Number of instances : 12969



 > TRAINING (2024-04-13 09:45:19) 


 | > Preprocessing samples
 | > Max text length: 188
 | > Min text length: 13
 | > Avg text length: 100.90014650319993
 | 
 | > Max audio length: 222643.0
 | > Min audio length: 24499.0
 | > Avg audio length: 144984.29755570978
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.



   --> TIME: 2024-04-13 09:45:32 -- STEP: 0/406 -- GLOBAL_STEP: 0
     | > current_lr: 2.5e-07 
     | > step_time: 1.1599  (1.1599225997924805)
     | > loader_time: 12.6947  (12.694692134857178)

 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.

   --> TIME: 2024-04-13 09:45:40 -- STEP: 25/406 -- GLOBAL_STEP: 25
     | > loss: 3.9253783226013184  (3.81333114306132

...


   --> TIME: 2024-04-13 14:28:34 -- STEP: 237/406 -- GLOBAL_STEP: 40025
     | > loss: -0.13221123814582825  (-0.13810097114949288)
     | > log_mle: -0.3471163511276245  (-0.3269686376998194)
     | > loss_dur: 0.21490511298179626  (0.18886766655032647)
     | > amp_scaler: 4096.0  (4096.0)
     | > grad_norm: tensor(31.6172, device='cuda:0')  (tensor(34.1840, device='cuda:0'))
     | > current_lr: 2.45e-05 
     | > step_time: 0.353  (0.2665157086738554)
     | > loader_time: 0.004  (0.009036178830303719)


   --> TIME: 2024-04-13 14:28:43 -- STEP: 262/406 -- GLOBAL_STEP: 40050
     | > loss: -0.12780824303627014  (-0.13860100627172994)
     | > log_mle: -0.32798540592193604  (-0.3282430444964926)
     | > loss_dur: 0.2001771628856659  (0.18964203822476267)
     | > amp_scaler: 4096.0  (4096.0)
     | > grad_norm: tensor(36.8432, device='cuda:0')  (tensor(35.3033, device='cuda:0'))
     | > current_lr: 2.45e-05 
     | > step_time: 0.3619  (0.27569705384378207)
     | > loader_time:

 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.17417970299720764 (-0.006696134805679321)
     | > avg_loss: -0.16577791422605515 (+0.0007441248744726181)
     | > avg_log_mle: -0.35437817871570587 (-0.0027471184730529785)
     | > avg_loss_dur: 0.18860026448965073 (+0.0034912433475255966)


 > EPOCH: 99/100
 --> c:/Users/wkaic/GitHub/Cockatoo.AI/experiments/model_c/output\run-April-13-2024_09+45AM-046eb8b

 > TRAINING (2024-04-13 14:30:10) 

   --> TIME: 2024-04-13 14:30:23 -- STEP: 6/406 -- GLOBAL_STEP: 40200
     | > loss: -0.18063248693943024  (-0.16544577976067862)
     | > log_mle: -0.31555676460266113  (-0.3182462652524312)
     | > loss_dur: 0.1349242776632309  (0.15280048549175262)
     | > amp_scaler: 4096.0  (4096.0)
     | > grad_norm: tensor(11.9370, device='cuda:0')  (tensor(24.6279, device='cuda:0'))
     | > current_lr: 2.475e-05 
     | > step_time: 0.171  (0.18100074927012125)
     | > loader_time: 0.001  (0.2378532886505127)


   --> TIME: 2024-04-13 14:30:28 -- 

 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.17894962430000305 (+0.00476992130279541)
     | > avg_loss: -0.15775873884558678 (+0.008019175380468369)
     | > avg_log_mle: -0.34596972167491913 (+0.008408457040786743)
     | > avg_loss_dur: 0.18821098282933235 (-0.00038928166031837463)

